# 掌握LCEL(LangChain Expression Language)

## 测试大模型

In [1]:
import os
from dotenv import load_dotenv
from langchain_deepseek import ChatDeepSeek
from langchain_core.messages import HumanMessage

# 加载环境变量
load_dotenv()

def call_deepseek(prompt):
    try:
        # 初始化 DeepSeek 聊天模型
        model = ChatDeepSeek(
            model="deepseek-chat",  # 模型名称
            api_key=os.getenv("DEEPSEEK_API_KEY"),  # 从环境变量获取 API 密钥
            api_base=os.getenv("DEEPSEEK_BASE_URL"),  # 从环境变量获取 API 基础地址
            temperature=0.7,  # 采样温度
            max_tokens=1000  # 最大生成 tokens
        )
        
        # 构造消息列表（LangChain 通常使用消息列表格式）
        messages = [HumanMessage(content=prompt)]
        
        # 调用模型并获取结果
        response = model.invoke(messages)
        
        # 返回生成的内容
        return response.content.strip()
    except Exception as e:
        return f"调用失败：{str(e)}"

# 测试调用
if __name__ == "__main__":
    prompt = "你好"
    result = call_deepseek(prompt)
    print("大模型回复：\n", result)

/mnt/mydisk/home/chenxd/.conda/envs/lang/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


大模型回复：
 你好！很高兴见到你！😊 我是DeepSeek，由深度求索公司创造的AI助手。无论你有什么问题、需要什么帮助，或者只是想聊聊天，我都很乐意为你提供支持！

我可以帮你解答各种问题，协助处理文档，进行创作和分析等等。有什么我可以为你做的吗？我会尽我所能热情地帮助你！✨


In [2]:
def stream_deepseek(prompt):
    model = ChatDeepSeek(
        model="deepseek-chat",
        api_key=os.getenv("DEEPSEEK_API_KEY"),
        api_base=os.getenv("DEEPSEEK_BASE_URL"),
        temperature=0.7
    )
    messages = [HumanMessage(content=prompt)]
    for chunk in model.stream(messages):
        print(chunk.content, end="", flush=True)

# 流式调用测试
if __name__ == "__main__":
    stream_deepseek("你好")

你好！很高兴见到你！😊 我是DeepSeek，由深度求索公司创造的AI助手。无论你有什么问题、需要什么帮助，或者只是想聊聊天，我都很乐意为你提供支持！

我可以帮你处理各种任务，比如：
- 回答问题和解释概念
- 协助写作和翻译
- 分析和处理文档
- 编程和技术支持
- 创意思考和头脑风暴

有什么我可以为你做的吗？请随时告诉我你的需求！✨

## Model I/O 三元组

In [ ]:
import os
import sys
import subprocess
import importlib
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI  # DeepSeek 兼容 OpenAI 接口

# 加载你的 .env
load_dotenv()

# 替换为 DeepSeek 的 API 配置
api_key = os.getenv("DEEPSEEK_API_KEY")
base_url = os.getenv("DEEPSEEK_BASE_URL")

# 1) Model：DeepSeek 模型（使用兼容 OpenAI 接口的方式）
llm = ChatOpenAI(
    model="deepseek-chat",  # DeepSeek 主推的对话模型
    temperature=0.2,
    api_key=api_key,
    base_url=base_url,
    max_tokens=1024,  # 可选：设置最大生成token数
    timeout=30,       # 可选：设置超时时间
)

# 2) Prompt：聊天提示模板（保持不变，DeepSeek 适配标准的 ChatML 格式）
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一位简洁、准确的中文助手。"),
    ("human", "请用一句话回答：{question}")
])

# 3) Output Parser：将模型输出解析为字符串
parser = StrOutputParser()

# 三元组链：Prompt → Model → Parser
chain = prompt | llm | parser

# 运行示例
inputs = {"question": "LangChain 的 model I/O 三元组是什么？"}
result = chain.invoke(inputs)
print("模型输出：", result)

# 可选：批量调用示例
# questions = [
#     {"question": "LangChain 的核心优势是什么？"},
#     {"question": "什么是 Prompt Engineering？"},
#     {"question": "LLM 的上下文窗口是什么？"}
# ]
# for input_data in questions:
#     print(f"\n问题：{input_data['question']}")
#     print(f"回答：{chain.invoke(input_data)}")

模型输出： LangChain 的 model I/O 三元组是输入（Prompts）、模型（Models）和输出解析（Output Parsers）。


## Runnable对象

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

load_dotenv()
api_key = os.getenv("DEEPSEEK_API_KEY")
base_url = os.getenv("DEEPSEEK_BASE_URL")

# 模型：deepseek-chat（使用 DeepSeek 的接口）
llm = ChatOpenAI(
    model="deepseek-chat",
    api_key=api_key,
    base_url=base_url,
    temperature=0.2,
)

# Prompt：简单聊天模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一位简洁、准确的中文助手。"),
    ("human", "请用一句话回答：{question}")
])

# 解析器：将消息解析为纯字符串
parser = StrOutputParser()

# 三元组链：Prompt → Model → Parser
chain = prompt | llm | parser

# 1) 直接调用 invoke
print(chain.invoke({"question": "用一句话解释 LangChain 的 Runnable。"}))

# 2) 使用 RunnableLambda 做输入变换，再接三元组链
to_question = RunnableLambda(lambda x: {"question": f"请分三点概括：{x['topic']}"})
chain2 = to_question | prompt | llm | parser

print(chain2.invoke({"topic": "Runnable 的核心能力"}))

# 3) 批量调用 batch
inputs = [
    {"topic": "Runnable 支持 invoke/batch/stream"},
    {"topic": "Runnable 通过 | 轻松组合组件"},
]
for out in chain2.batch(inputs):
    print(out)

LangChain 的 Runnable 是一个标准化接口，用于统一调用链、工具、模型等组件，简化构建和组合 AI 应用的过程。
Runnable 的核心能力包括：任务编排、工具调用和统一接口。
Runnable 支持通过 invoke 执行单次调用、batch 处理批量任务以及 stream 实现流式响应。
Runnable 通过管道符 | 轻松实现组件串联、并行与条件组合。


## 通过 Runnable 输出链路 ASCII 图

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 直接加载当前目录下的 .env
load_dotenv()

# 读取 DeepSeek 环境变量
api_key = os.getenv("DEEPSEEK_API_KEY")
base_url = os.getenv("DEEPSEEK_BASE_URL")

# 模型：DeepSeek 的 OpenAI 兼容接口
llm = ChatOpenAI(
    model="deepseek-chat",
    api_key=api_key,
    base_url=base_url,
    temperature=0.2,
)

# Prompt：简单聊天模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一位简洁、准确的中文助手。"),
    ("human", "请用一句话回答：{question}")
])

# 解析器：将模型消息解析为字符串
parser = StrOutputParser()

# 三元组链（Runnable）：Prompt → Model → Parser
chain = prompt | llm | parser

# 输出链路结构的 ASCII 字符图
chain.get_graph().print_ascii()

# 可选：实际调用一次链，验证逻辑（不是必须）
# print(chain.invoke({"question": "LangChain 的 Runnable 是什么？"}))

/mnt/mydisk/home/chenxd/.conda/envs/lang/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
